In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import pylab as plt

# Project

### Description

This is a learning project. The point of it is to run analysis on the HoI4 air combat algorithm. It will use python and MatPlotLib to make the calculations and display them as graphs to make analysis and comparison easier.

### Preparation

We need to set up the basic data that we will be running the algorithm over to see the results. We will be using objects to store and pass data. 

In [9]:
class Plane:
    def __init__(self, attack, defence, agility, speed, cost):
        self.attack = attack
        self.defence = defence
        self.agility = agility
        self.speed = speed
        self.cost = cost


This is the primary class we will be using. The properties are the values for the planes that we will be comparing.

In [15]:
class VariantPlane(Plane):
    def __init__(self, Plane):
        super().__init__(Plane.attack, Plane,defence, Plane.agility, Plane.speed, Plane.cost)
        self.weapons = 0 # +10% attack, - 1% speed, 10% Reliability, 5% agility
        self.range = 0 # +10% Range, -10% Reliability
        self.engine = 0 # +2% speed, 5% agility
        self.reliability = 0 # +10% Reliabilty

I will come back to figure out this later. The point is that I want to make variants to planes without having to type in everything. I will send in the plane I want a variant of and then overwrite the specific values.

I see that this might not be the best way to do this with this specific dataset. The upgrades are percentage bonus/penalties. That means that I want the original plane values and the upgrade points, and afterwards apply the bonuses.

I also want to be able to only set one of the possible variant values, and have the others initiate as Zero

In [ ]:
def damage(attacker, defender):
    return